# How to on Spark: Forecasting
> Run TimeGPT distributedly on top of Spark.

`TimeGPT` works on top of Spark, Dask, and Ray through Fugue. `TimeGPT` will read the input DataFrame and use the corresponding engine. For example, if the input is a Spark DataFrame, `TimeGPT` will use the existing Spark session to run the forecast.


In [ ]:
#| hide
from nixtlats.utils import colab_badge

In [ ]:
#| echo: false
colab_badge('docs/how-to-guides/0_distributed_fcst_spark')

# Installation 

As long as Spark is installed and configured, `TimeGPT` will be able to use it. If executing on a distributed Spark cluster, make use the `nixtlats` library is installed across all the workers.

## Executing on Spark

To run the forecasts distributed on Spark, just pass in a Spark DataFrame instead. 

In [ ]:
#| hide
import os

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

Instantiate `NixtlaClient` class.

In [ ]:
from nixtlats import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'my_api_key_provided_by_nixtla'
)

In [ ]:
#| hide
nixtla_client = NixtlaClient()

Use Spark as an engine.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

### Forecast

In [ ]:
url_df = 'https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short.csv'
spark_df = spark.createDataFrame(pd.read_csv(url_df))
spark_df.show(5)

In [ ]:
fcst_df = nixtla_client.forecast(spark_df, h=12)
fcst_df.show(5)

In [ ]:
#| hide
from fastcore.test import test_fail

In [ ]:
#| hide
# test different results for different models
fcst_df_1 = fcst_df.toPandas()
fcst_df_2 = nixtla_client.forecast(spark_df, h=12, model='timegpt-1-long-horizon')
fcst_df_2 = fcst_df_2.toPandas()
test_fail(
    lambda: pd.testing.assert_frame_equal(fcst_df_1[['TimeGPT']], fcst_df_2[['TimeGPT']]),
    contains='(column name="TimeGPT") are different'
)

### Forecast with exogenous variables

Exogenous variables or external factors are crucial in time series forecasting as they provide additional information that might influence the prediction. These variables could include holiday markers, marketing spending, weather data, or any other external data that correlate with the time series data you are forecasting.

For example, if you're forecasting ice cream sales, temperature data could serve as a useful exogenous variable. On hotter days, ice cream sales may increase.

To incorporate exogenous variables in TimeGPT, you'll need to pair each point in your time series data with the corresponding external data.

Let's see an example.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-with-ex-vars.csv')
spark_df = spark.createDataFrame(df)
spark_df.show(5)

To produce forecasts we have to add the future values of the exogenous variables. Let's read this dataset. In this case we want to predict 24 steps ahead, therefore each unique id will have 24 observations.

In [ ]:
future_ex_vars_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/electricity-short-future-ex-vars.csv')
spark_future_ex_vars_df = spark.createDataFrame(future_ex_vars_df)
spark_future_ex_vars_df.show(5)

Let's call the `forecast` method, adding this information:

In [ ]:
timegpt_fcst_ex_vars_df = nixtla_client.forecast(df=spark_df, X_df=spark_future_ex_vars_df, h=24, level=[80, 90])
timegpt_fcst_ex_vars_df.show(5)

In [ ]:
spark.stop()